# Goal
The goal of this notebook is to compare the performance of multi-objective probability of improvement backed by a multi-
objective homogeneous random forest to random search.

# Methodology
We only need to allow each strategy to execute for some fixed number of iterations. We can subsequently process the resulting observations to estimate the size of the pareto frontier over time. 

Specifically, let's first run both strategies on a 2D hypersphere, 10 times each and compare their average performance.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionConfigStore import objective_function_config_store
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere

from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier

from mlos.Spaces import Point

objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_2_mutually_exclusive_polynomials")
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)

In [3]:
random_params_df = objective_function.parameter_space.random_dataframe(10)
random_params_df

,polynomial_id,domain_1.x_0,domain_1.x_1,domain_0.x_0,domain_0.x_1
0,1,-997.417803,490.113850,NaN,NaN
1,1,668.515787,-79.931971,NaN,NaN
2,1,881.202802,-966.482091,NaN,NaN
3,0,NaN,NaN,394.492201,-119.614950
4,0,NaN,NaN,270.519358,-497.251643
5,1,-818.063896,-629.021691,NaN,NaN
6,1,-433.853602,-895.929924,NaN,NaN
7,1,-185.702147,381.386867,NaN,NaN
8,1,329.636606,-871.975085,NaN,NaN
9,0,NaN,NaN,-476.934168,-929.759039


In [4]:
objective_function.evaluate_dataframe(random_params_df)

,y0,y1
0,1.181166e+06,-5.477513e+06
1,-6.958022e+05,-3.247003e+06
2,2.652801e+06,-3.601524e+06
3,-1.239998e+06,-5.461559e+04
4,-1.102360e+06,-7.449382e+05
5,-8.248551e+06,-1.015892e+07
6,-7.288583e+06,-6.813047e+06
7,4.434460e+04,-2.841781e+05
8,-3.758329e+05,-1.472037e+06
9,-5.739424e+06,-1.026666e+07


In [5]:
optimization_problem = objective_function.default_optimization_problem

optimizer_config = bayesian_optimizer_config_store.get_config_by_name("default_multi_objective_optimizer_config")

In [6]:
objectives_dfs_by_run = []
optimizers_by_run = []
num_iterations_per_run = 1000
num_runs = 10

for run_id in range(num_runs):

    optimizer = BayesianOptimizerFactory().create_local_optimizer(
        optimization_problem=optimization_problem,
        optimizer_config=optimizer_config
    )
    
    optimizers_by_run.append(optimizer)

    for i in range(num_iterations_per_run):
        print(f"[{run_id}/{num_runs}][{i+1}/{num_iterations_per_run}]")
        suggestion = optimizer.suggest()
        values = objective_function.evaluate_point(suggestion)
        optimizer.register(suggestion.to_dataframe(), values.to_dataframe())
    
    _, objectives_df, _ = optimizer.get_all_observations()
    objectives_dfs_by_run.append(objectives_df)
              

01/23/2021 07:19:03 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 106 -        _create_estimators() ] Creating 10 estimators. Tree config: {
  "criterion": "mse",
  "splitter": "best",
  "max_depth": 0,
  "min_samples_split": 2,
  "min_samples_leaf": 3,
  "min_weight_fraction_leaf": 0,
  "max_features": "auto",
  "max_leaf_nodes": 0,
  "min_impurity_decrease": 0,
  "ccp_alpha": 0,
  "min_samples_to_fit": 10,
  "n_new_samples_before_refit": 10
}. Request id: 0.30259741115742655
01/23/2021 07:19:03 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_0_input_space
  Dimensions:
    domain___polynomial_id: {0, 1}
    contains_context: {False}
    domain___domain_1___x_1: [-1024.00, 1024.00]
    domain___domain_1___x_0: [-1024.00, 1024.00]
01/23/2021 07:19:03 -   BayesianOptimizerFactory -    INFO - [Homogen

01/23/2021 07:19:03 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_8_input_space
  Dimensions:
    domain___polynomial_id: {0, 1}
    domain___domain_0___x_1: [-1024.00, 1024.00]
    domain___domain_0___x_0: [-1024.00, 1024.00]
    contains_context: {False}
01/23/2021 07:19:03 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_9_input_space
  Dimensions:
    domain___domain_0___x_1: [-1024.00, 1024.00]
    contains_context: {False}
    domain___polynomial_id: {0, 1}
    domain___domain_0___x_0: [-1024.00, 1024.00]


[0/10][1/1000]
[0/10][2/1000]
[0/10][3/1000]
[0/10][4/1000]
[0/10][5/1000]
[0/10][6/1000]
[0/10][7/1000]
[0/10][8/1000]
[0/10][9/1000]
[0/10][10/1000]
[0/10][11/1000]
[0/10][12/1000]
[0/10][13/1000]
[0/10][14/1000]
[0/10][15/1000]
[0/10][16/1000]
[0/10][17/1000]
[0/10][18/1000]
[0/10][19/1000]
[0/10][20/1000]
[0/10][21/1000]
[0/10][22/1000]
[0/10][23/1000]
[0/10][24/1000]
[0/10][25/1000]
[0/10][26/1000]
[0/10][27/1000]
[0/10][28/1000]
[0/10][29/1000]
[0/10][30/1000]
[0/10][31/1000]
[0/10][32/1000]
[0/10][33/1000]
[0/10][34/1000]
[0/10][35/1000]
[0/10][36/1000]
[0/10][37/1000]
[0/10][38/1000]
[0/10][39/1000]
[0/10][40/1000]
[0/10][41/1000]
[0/10][42/1000]
[0/10][43/1000]
[0/10][44/1000]
[0/10][45/1000]
[0/10][46/1000]
[0/10][47/1000]
[0/10][48/1000]
[0/10][49/1000]
[0/10][50/1000]
[0/10][51/1000]
[0/10][52/1000]
[0/10][53/1000]
[0/10][54/1000]
[0/10][55/1000]
[0/10][56/1000]
[0/10][57/1000]
[0/10][58/1000]
[0/10][59/1000]
[0/10][60/1000]
[0/10][61/1000]
[0/10][62/1000]
[0/10][63/1000]
[

[0/10][492/1000]
[0/10][493/1000]
[0/10][494/1000]
[0/10][495/1000]
[0/10][496/1000]
[0/10][497/1000]
[0/10][498/1000]
[0/10][499/1000]
[0/10][500/1000]
[0/10][501/1000]
[0/10][502/1000]
[0/10][503/1000]
[0/10][504/1000]
[0/10][505/1000]
[0/10][506/1000]
[0/10][507/1000]
[0/10][508/1000]
[0/10][509/1000]
[0/10][510/1000]
[0/10][511/1000]
[0/10][512/1000]
[0/10][513/1000]
[0/10][514/1000]
[0/10][515/1000]
[0/10][516/1000]
[0/10][517/1000]
[0/10][518/1000]
[0/10][519/1000]
[0/10][520/1000]
[0/10][521/1000]
[0/10][522/1000]
[0/10][523/1000]
[0/10][524/1000]
[0/10][525/1000]
[0/10][526/1000]
[0/10][527/1000]
[0/10][528/1000]
[0/10][529/1000]
[0/10][530/1000]
[0/10][531/1000]
[0/10][532/1000]
[0/10][533/1000]
[0/10][534/1000]
[0/10][535/1000]
[0/10][536/1000]
[0/10][537/1000]
[0/10][538/1000]
[0/10][539/1000]
[0/10][540/1000]
[0/10][541/1000]
[0/10][542/1000]
[0/10][543/1000]
[0/10][544/1000]
[0/10][545/1000]
[0/10][546/1000]
[0/10][547/1000]
[0/10][548/1000]
[0/10][549/1000]
[0/10][550/100

[0/10][974/1000]
[0/10][975/1000]
[0/10][976/1000]
[0/10][977/1000]
[0/10][978/1000]
[0/10][979/1000]
[0/10][980/1000]
[0/10][981/1000]
[0/10][982/1000]
[0/10][983/1000]
[0/10][984/1000]
[0/10][985/1000]
[0/10][986/1000]
[0/10][987/1000]
[0/10][988/1000]
[0/10][989/1000]
[0/10][990/1000]
[0/10][991/1000]
[0/10][992/1000]
[0/10][993/1000]
[0/10][994/1000]
[0/10][995/1000]
[0/10][996/1000]
[0/10][997/1000]
[0/10][998/1000]
[0/10][999/1000]


...

01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_7_input_space
  Dimensions:
    domain___polynomial_id: {0, 1}
    domain___domain_0___x_1: [-1024.00, 1024.00]
    contains_context: {False}
    domain___domain_0___x_0: [-1024.00, 1024.00]
01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_7_input_space
  Dimensions:
    domain___polynomial_id: {0, 1}
    domain___domain_0___x_1: [-1024.00, 1024.00]
    contains_context: {False}
    domain___domain_0___x_0: [-1024.00, 1024.00]
01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel w

01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_9_input_space
  Dimensions:
    contains_context: {False}
    domain___domain_1___x_0: [-1024.00, 1024.00]
    domain___domain_1___x_1: [-1024.00, 1024.00]
    domain___polynomial_id: {0, 1}
01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_9_input_space
  Dimensions:
    contains_context: {False}
    domain___domain_1___x_0: [-1024.00, 1024.00]
    domain___domain_1___x_1: [-1024.00, 1024.00]
    domain___polynomial_id: {0, 1}
01/23/2021 10:16:54 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel w

[9/10][1/1000]
[9/10][2/1000]
[9/10][3/1000]
[9/10][4/1000]
[9/10][5/1000]
[9/10][6/1000]
[9/10][7/1000]
[9/10][8/1000]
[9/10][9/1000]
[9/10][10/1000]
[9/10][11/1000]
[9/10][12/1000]
[9/10][13/1000]
[9/10][14/1000]
[9/10][15/1000]
[9/10][16/1000]
[9/10][17/1000]
[9/10][18/1000]
[9/10][19/1000]
[9/10][20/1000]
[9/10][21/1000]
[9/10][22/1000]
[9/10][23/1000]
[9/10][24/1000]
[9/10][25/1000]
[9/10][26/1000]
[9/10][27/1000]
[9/10][28/1000]
[9/10][29/1000]
[9/10][30/1000]
[9/10][31/1000]
[9/10][32/1000]
[9/10][33/1000]
[9/10][34/1000]
[9/10][35/1000]
[9/10][36/1000]
[9/10][37/1000]
[9/10][38/1000]
[9/10][39/1000]
[9/10][40/1000]
[9/10][41/1000]
[9/10][42/1000]
[9/10][43/1000]
[9/10][44/1000]
[9/10][45/1000]
[9/10][46/1000]
[9/10][47/1000]
[9/10][48/1000]
[9/10][49/1000]
[9/10][50/1000]
[9/10][51/1000]
[9/10][52/1000]
[9/10][53/1000]
[9/10][54/1000]
[9/10][55/1000]
[9/10][56/1000]
[9/10][57/1000]
[9/10][58/1000]
[9/10][59/1000]
[9/10][60/1000]
[9/10][61/1000]
[9/10][62/1000]
[9/10][63/1000]
[

[9/10][490/1000]
[9/10][491/1000]
[9/10][492/1000]
[9/10][493/1000]
[9/10][494/1000]
[9/10][495/1000]
[9/10][496/1000]
[9/10][497/1000]
[9/10][498/1000]
[9/10][499/1000]
[9/10][500/1000]
[9/10][501/1000]
[9/10][502/1000]
[9/10][503/1000]
[9/10][504/1000]
[9/10][505/1000]
[9/10][506/1000]
[9/10][507/1000]
[9/10][508/1000]
[9/10][509/1000]
[9/10][510/1000]
[9/10][511/1000]
[9/10][512/1000]
[9/10][513/1000]
[9/10][514/1000]
[9/10][515/1000]
[9/10][516/1000]
[9/10][517/1000]
[9/10][518/1000]
[9/10][519/1000]
[9/10][520/1000]
[9/10][521/1000]
[9/10][522/1000]
[9/10][523/1000]
[9/10][524/1000]
[9/10][525/1000]
[9/10][526/1000]
[9/10][527/1000]
[9/10][528/1000]
[9/10][529/1000]
[9/10][530/1000]
[9/10][531/1000]
[9/10][532/1000]
[9/10][533/1000]
[9/10][534/1000]
[9/10][535/1000]
[9/10][536/1000]
[9/10][537/1000]
[9/10][538/1000]
[9/10][539/1000]
[9/10][540/1000]
[9/10][541/1000]
[9/10][542/1000]
[9/10][543/1000]
[9/10][544/1000]
[9/10][545/1000]
[9/10][546/1000]
[9/10][547/1000]
[9/10][548/100

[9/10][973/1000]
[9/10][974/1000]
[9/10][975/1000]
[9/10][976/1000]
[9/10][977/1000]
[9/10][978/1000]
[9/10][979/1000]
[9/10][980/1000]
[9/10][981/1000]
[9/10][982/1000]
[9/10][983/1000]
[9/10][984/1000]
[9/10][985/1000]
[9/10][986/1000]
[9/10][987/1000]
[9/10][988/1000]
[9/10][989/1000]
[9/10][990/1000]
[9/10][991/1000]
[9/10][992/1000]
[9/10][993/1000]
[9/10][994/1000]
[9/10][995/1000]
[9/10][996/1000]
[9/10][997/1000]
[9/10][998/1000]
[9/10][999/1000]
[9/10][1000/1000]


In [7]:
pareto_volumes_over_time_per_run = [[] for i in range(10, 1001, 10)]
for run_id, objectives_df in enumerate(objectives_dfs_by_run):
    for i, j in enumerate(range(10, 1001, 10)):
        pareto_frontier = ParetoFrontier(optimization_problem=optimization_problem, objectives_df=objectives_df[:j])
        pareto_volume_estimator = pareto_frontier.approximate_pareto_volume(num_samples=1000000)
        lower_bound, upper_bound = pareto_volume_estimator.get_two_sided_confidence_interval_on_pareto_volume(alpha=0.05)
        pareto_volume = (lower_bound + upper_bound) / 2
        pareto_volumes_over_time_per_run[i].append(pareto_volume)


In [8]:
average_pareto_volume_by_iteration = [sum(pareto_volumes) / len(pareto_volumes) for pareto_volumes in pareto_volumes_over_time_per_run]

In [16]:
pareto_volumes_per_run_over_time = [[0 for i in range(10, 1001, 10)] for objectives_df in objectives_dfs_by_run]
for run_id, objectives_df in enumerate(objectives_dfs_by_run):
    for col, iteration_number in enumerate(range(10, 1001, 10)):
        pareto_volumes_per_run_over_time[run_id][col] = pareto_volumes_over_time_per_run[col][run_id]

In [9]:
average_pareto_volume_by_iteration

[103549850154832.95,
 123300736764074.22,
 133488183930641.05,
 161354557423176.25,
 165844657027089.5,
 172204121846934.66,
 193406000535543.84,
 199968503464227.7,
 221393861024101.34,
 222218305272933.6,
 227224388559660.94,
 232441293030765.9,
 232440133865522.7,
 233722378237646.3,
 233722295644579.8,
 234321951846886.44,
 234322078066281.8,
 234319379438322.3,
 234320810390673.66,
 234895300671175.9,
 234895952641438.1,
 234886894451494.84,
 234889138381174.44,
 234888354458831.9,
 234888438368950.2,
 234895535838375.06,
 234895714335986.84,
 234890364829189.1,
 234893962049139.0,
 234892468269020.0,
 234894263230023.1,
 234890717034497.0,
 234893826345057.3,
 236461487253622.0,
 236465963106527.66,
 240412884438192.7,
 242363410868996.9,
 242364248984358.34,
 242363445805368.94,
 243054530402902.2,
 243051994108731.84,
 243050524562732.7,
 243055458576403.84,
 243504740801865.66,
 243868156136705.2,
 243859889523933.7,
 243862153346830.16,
 251205931953459.1,
 251202320537095.25

In [10]:
# Now let's do the random ones :)
#
num_runs = 10
objectives_dfs_by_random_run = []

for run_id in range(num_runs):
    suggestions_df = objective_function.parameter_space.random_dataframe(num_iterations_per_run)
    objectives_df = objective_function.evaluate_dataframe(suggestions_df)
    objectives_dfs_by_random_run.append(objectives_df)
    
pareto_volumes_over_time_per_random_run = [[] for i in range(10, 1001, 10)]
for run_id, objectives_df in enumerate(objectives_dfs_by_random_run):
    for i, j in enumerate(range(10, 1001, 10)):
        pareto_frontier = ParetoFrontier(optimization_problem=optimization_problem, objectives_df=objectives_df[:j])
        pareto_volume_estimator = pareto_frontier.approximate_pareto_volume(num_samples=1000000)
        lower_bound, upper_bound = pareto_volume_estimator.get_two_sided_confidence_interval_on_pareto_volume(alpha=0.05)
        pareto_volume = (lower_bound + upper_bound) / 2
        pareto_volumes_over_time_per_random_run[i].append(pareto_volume)

average_random_pareto_volume_by_iteration = [sum(pareto_volumes) / len(pareto_volumes) for pareto_volumes in pareto_volumes_over_time_per_random_run]

In [19]:
pareto_volumes_per_random_run_over_time = [[0 for i in range(10, 1001, 10)] for run_id in range(num_runs)]
for run_id in range(num_runs):
    for col, iteration in enumerate(range(10, 1001, 10)):
        pareto_volumes_per_random_run_over_time[run_id][col] = pareto_volumes_over_time_per_random_run[col][run_id]

In [11]:
average_random_pareto_volume_by_iteration

[77650108828674.1,
 145704905541358.2,
 159652084506063.28,
 165279799782538.78,
 166195518079992.16,
 168907086472081.44,
 173687000544758.7,
 181719100169176.9,
 186590986570427.7,
 200358554607962.16,
 204092971336188.34,
 214972607507526.9,
 219367197578374.16,
 219368045310104.4,
 232525980688829.0,
 234106262688599.94,
 235202137643453.0,
 235203957537580.75,
 236030113594533.3,
 238505893422910.8,
 239640866295973.56,
 239641755687552.16,
 239655460818098.66,
 239655894530103.3,
 239658221667789.5,
 240017181818222.8,
 240893924856875.9,
 240885902894835.8,
 243643121455375.25,
 251500314786152.44,
 251501706712922.6,
 251585613167517.4,
 251584856100485.9,
 251580956922413.4,
 251585650082544.8,
 251583569826609.9,
 251578683720025.9,
 251584908216141.84,
 251583010533800.6,
 251592862504604.16,
 251585900332466.84,
 251592396701852.16,
 251584790892308.56,
 251590862660724.06,
 251585646085170.2,
 251587483354367.16,
 251581232577711.66,
 251592142880296.34,
 251770778838363.2

In [20]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[i for i in range(10, 1001, 10)],
    y=average_pareto_volume_by_iteration,
    name='Guided Optimization: Average Pareto Volume vs. Iteration'
))

fig.add_trace(go.Scatter(
    x=[i for i in range(10, 1001, 10)],
    y=average_random_pareto_volume_by_iteration,
    name='Random Search: Average Pareto Volume vs. Iteration'
))

for run_id, guided_pareto_volume in enumerate(pareto_volumes_per_run_over_time):
    fig.add_trace(go.Scatter(
    x=[i for i in range(10, 1001, 10)],
    y=guided_pareto_volume,
    name=f'Guided Run {run_id}'
))

for run_id, guided_pareto_volume in enumerate(pareto_volumes_per_random_run_over_time):
    fig.add_trace(go.Scatter(
    x=[i for i in range(10, 1001, 10)],
    y=guided_pareto_volume,
    name=f'Random Run {run_id}'
))
    
fig.show()